# Logging using the MLFlow Logger

This notebook demonstrates how to use the logging function for MLFLow

## Installation

You can install everything from the command line using the following commands.

### Installing Anomalib

The easiest way to install anomalib is to use pip.

In [ ]:
%pip install anomalib

### Install Anomalib and MLFlow

Install anomalib with MLFlow using pip.

In [ ]:
%pip install anomalib[loggers]

### Install MLFlow

Install MLFlow using pip.

In [ ]:
%pip install mlflow

# Run MLFlow Server

You can execute the following command in a seperate terminal to access the MLFlow UI. Or you can come back to the flllowing cell and execute it.

WARNING: This cell needs to be interrupted manually to continue!

In [ ]:
!mlflow server

## Dataset Directory

This cell is to ensure we change the directory to have access to the datasets.

This part is borrowed from the datamodule mvtec notebook.

In [1]:
from pathlib import Path

# NOTE: Provide the path to the dataset root directory.
#   If the datasets is not downloaded, it will be downloaded
#   to this directory.
dataset_root = Path.cwd().parent / "datasets" / "MVTec"

## Imports

In [2]:
from anomalib.data import MVTec
from anomalib import TaskType
from anomalib.callbacks.checkpoint import ModelCheckpoint
from lightning.pytorch.callbacks import EarlyStopping
from anomalib.models import Fastflow
from anomalib.loggers import AnomalibMLFlowLogger
from anomalib.engine import Engine

import matplotlib.pyplot as plt

## Data Module

Using the data module to load the MVTec dataset. But first let's print the docstring.

In [3]:
help(MVTec)

Help on class MVTec in module anomalib.data.image.mvtec:

class MVTec(anomalib.data.base.datamodule.AnomalibDataModule)
 |  MVTec(root: pathlib.Path | str = './datasets/MVTec', category: str = 'bottle', train_batch_size: int = 32, eval_batch_size: int = 32, num_workers: int = 8, task: anomalib.TaskType = <TaskType.SEGMENTATION: 'segmentation'>, image_size: tuple[int, int] | None = None, transform: torchvision.transforms.v2._transform.Transform | None = None, train_transform: torchvision.transforms.v2._transform.Transform | None = None, eval_transform: torchvision.transforms.v2._transform.Transform | None = None, test_split_mode: anomalib.data.utils.split.TestSplitMode = <TestSplitMode.FROM_DIR: 'from_dir'>, test_split_ratio: float = 0.2, val_split_mode: anomalib.data.utils.split.ValSplitMode | str = <ValSplitMode.SAME_AS_TEST: 'same_as_test'>, val_split_ratio: float = 0.5, seed: int | None = None) -> None
 |  
 |  MVTec Datamodule.
 |  
 |  Args:
 |      root (Path | str): Path to the 

In [4]:
datamodule = MVTec(
    root=dataset_root,
    category="bottle",
    image_size=256,
    train_batch_size=32,
    eval_batch_size=32,
    num_workers=24,
    task=TaskType.SEGMENTATION,
)

## Model

Setup Fastflow as an example model.

In [5]:
model = Fastflow(backbone="resnet18", flow_steps=8)

## MLFlow Logger

Setup the MLFlow logger. But first let's print the docstring.

In [6]:
help(AnomalibMLFlowLogger)

Help on class AnomalibMLFlowLogger in module anomalib.loggers.mlflow:

class AnomalibMLFlowLogger(anomalib.loggers.base.ImageLoggerBase, lightning.pytorch.loggers.mlflow.MLFlowLogger)
 |  AnomalibMLFlowLogger(experiment_name: str | None = 'anomalib_logs', run_name: str | None = None, tracking_uri: str | None = None, save_dir: str | None = './mlruns', log_model: Optional[Literal[True, False, 'all']] = False, prefix: str | None = '', **kwargs) -> None
 |  
 |  Logger for MLFlow
 |  
 |  Adds interface for ``add_image`` in the logger rather than calling the
 |  experiment object.
 |  
 |  .. note::
 |      Same as the MLFlowLogger provided by PyTorch Lightning and the doc string is reproduced below.
 |  
 |  Track your parameters, metrics, source code and more using
 |  `MLFlow <https://mlflow.org/#core-concepts>`_.
 |  
 |  Install it with pip:
 |  
 |  .. code-block:: bash
 |  
 |      pip install mlflow
 |  
 |  Args:
 |      experiment_name: The name of the experiment.
 |      run_nam

In [7]:
mlflow_logger = AnomalibMLFlowLogger()

## Training

### Callbacks

In [8]:
model_checkpoint = ModelCheckpoint(mode="max", monitor="pixel_AUROC")

early_stopping = EarlyStopping(monitor="pixel_AUROC", mode="max", patience=3)

### Setup Engine

In [9]:
callbacks = [
    model_checkpoint,
    early_stopping,
]

kwargs = {"log_every_n_steps": 3}

engine = Engine(
    callbacks=callbacks,
    pixel_metrics="AUROC",
    accelerator="auto",
    devices=1,
    logger=mlflow_logger, # Logger is set here
    **kwargs,
)

### Fit the Model

In [10]:
engine.fit(model=model, datamodule=datamodule)

/home/dominik/GitHub/Forked-Repos/anomalib/.conda/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `PrecisionRecallCurve` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
No implementation of `configure_transforms` was provided in the Lightning model. Using default transforms from the base class. This may not be suitable for your use case. Please override `configure_transforms` in your model.
/home/dominik/GitHub/Forked-Repos/anomalib/.conda/lib/python3.10/site-packages/lightning/pytorch/loops/utilities.py:73: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
You are using a CUDA device ('NVIDIA GeForce RTX 3090 Ti') that has Tensor Cores. To properly u

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

## Testing

In [11]:
engine.test(model=model, dataloaders=datamodule.test_dataloader())

/home/dominik/GitHub/Forked-Repos/anomalib/.conda/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `PrecisionRecallCurve` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │            1.0            │
│       image_F1Score       │            1.0            │
│        pixel_AUROC        │    0.9721677303314209     │
└───────────────────────────┴───────────────────────────┘

[{'pixel_AUROC': 0.9721677303314209, 'image_AUROC': 1.0, 'image_F1Score': 1.0}]

## Demo Track Figure

In [12]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.plot([0, 0], [2, 3])

mlflow_logger.add_image(fig, "figure_demo.png")

## Save Model to MLFlow

In [13]:
import mlflow

with mlflow.start_run(run_id=mlflow_logger.run_id):
    mlflow.pytorch.log_model(engine.model.model, "Fastflow")

/home/dominik/GitHub/Forked-Repos/anomalib/.conda/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


## Load Model from MLFlow

In [14]:
model_uri = f"runs:/{mlflow_logger.run_id}/Fastflow"
mlflow.pytorch.load_model(model_uri)

FastflowModel(
  (feature_extractor): FeatureListNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (drop_block): Identity()
        (act1): ReLU(inplace=True)
        (aa): Identity()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): ReLU(inplace=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stri